# 九州８県を結合

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.speedups
import pandas as pd
import os
import glob
%matplotlib inline

## （１）九州各県（市町村単位）の地図を作成

### 九州各県のファイルを取得

In [ ]:
files = [geojson for geojson in glob.glob('../Prefecture/*[4][0-9]*.geojson')]

In [ ]:
files

In [ ]:
gdf = pd.concat([gpd.read_file(file) for file in files])

In [ ]:
len(gdf)

In [ ]:
gdf.head()

### 自治体コードで融合

In [ ]:
gdf = gdf.dissolve(by="N03_007")

In [ ]:
len(gdf)

### ”所属未定地”の列を削除

In [ ]:
gdf = gdf[gdf["N03_004"] != '所属未定地']

In [ ]:
len(gdf)

### 不要な列を削除

In [ ]:
gdf = gdf.drop(["N03_002","N03_003"], axis=1)

In [ ]:
gdf = gdf.reset_index()

In [ ]:
gdf = gdf[["N03_001","N03_004","N03_007","geometry"]]

In [ ]:
gdf.plot()

### ファイルを出力

In [ ]:
outfp_1 = "../Prefecture_simplify/Kyushu/Kyushu.geojson"

In [ ]:
gdf.to_file(outfp_1,driver="GeoJSON")

## （２）作成した地図を簡素化

### 結合したファイルを取得

In [ ]:
gdf_1 = gpd.read_file(outfp_1)

In [ ]:
gdf_1.head()

### 簡素化
- QGIS – 簡素化
- http://taustation.com/qgis-geometry-simplification/

In [ ]:
# 許容範囲を設定
tol = 0.01

In [ ]:
gdf_1['geom_gen'] = gdf_1.simplify(tolerance=tol)

In [ ]:
gdf_1 = gdf_1.set_geometry('geom_gen')

In [ ]:
gdf_1 = gdf_1.drop(["geometry"], axis=1)

In [ ]:
gdf_1.head()

In [ ]:
gdf_1.plot()

### 簡素化したファイルを出力

In [ ]:
outfp_2 = "../Prefecture_simplify/Kyushu/Kyushu_Simplify_1_" + str(tol) + ".geojson"

In [ ]:
gdf_1.to_file(outfp_2,driver="GeoJSON")

### ファイルサイズを比較

In [ ]:
os.path.getsize(outfp_1)

In [ ]:
os.path.getsize(outfp_2)

## （３）九州各県（県単位）の地図を作成

### 簡素化前の九州地方の結合ファイルを取得

In [ ]:
url = "../Prefecture_simplify/Kyushu/Kyushu.geojson"

In [ ]:
gdf_2 = gpd.read_file(url)

In [ ]:
gdf_2.head()

### 県ごとに融合

In [ ]:
gdf_2 = gdf_2.dissolve(by="N03_001")

### 不要な列を削除

In [ ]:
gdf_2 = gdf_2.reset_index()

In [ ]:
gdf_2 = gdf_2.drop(["N03_004"], axis=1)

### 都道府県コード（2桁）を作成

In [ ]:
gdf_2["N03_007"] = gdf_2["N03_007"].apply(lambda x: x[:2])

In [ ]:
gdf_2 = gdf_2[["N03_001","N03_007","geometry"]]

In [ ]:
gdf_2

In [ ]:
gdf_2.plot()

### 簡素化

In [ ]:
gdf_2['geom_gen'] = gdf_2.simplify(tolerance=tol)

In [ ]:
gdf_2 = gdf_2.set_geometry('geom_gen')

In [ ]:
gdf_2 = gdf_2.drop(["geometry"], axis=1)

In [ ]:
gdf_2

In [ ]:
gdf_2.plot()

### ファイルを出力

In [ ]:
outfp_3 = "../Prefecture_simplify/Kyushu/Kyushu_Simplify_2_" + str(tol) + ".geojson"

In [ ]:
gdf_2.to_file(outfp_3,driver="GeoJSON")

### ファイルサイズを確認

In [ ]:
os.path.getsize(outfp_3)